<a href="https://colab.research.google.com/github/Pruthviraj141/Verifix/blob/main/ekyc_complete_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# 1. Install Tesseract OCR engine
!apt-get update && apt-get install -y tesseract-ocr

# 2. Install Python wrapper and image processing libraries
!pip install pytesseract opencv-python pillow numpy

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 2s (2,115 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1

In [18]:
import cv2
import numpy as np
import pytesseract
from PIL import Image
import re
from typing import Dict, Optional, Tuple

class EKYCExtractor:
    """Fast and efficient information extractor for Aadhaar and PAN cards"""

    def __init__(self):
        # Aadhaar card keywords for identification
        self.aadhaar_keywords = [
            'GOVERNMENT OF INDIA',
            'UNIQUE IDENTIFICATION',
            'UIDAI',
            'AADHAAR',
            'भारत सरकार'
        ]

        # PAN card keywords for identification
        self.pan_keywords = [
            'INCOME TAX DEPARTMENT',
            'PERMANENT ACCOUNT NUMBER',
            'PAN',
            'आयकर विभाग'
        ]

    def preprocess_fast(self, image_path: str) -> np.ndarray:
        """Fast preprocessing optimized for OCR"""
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError(f"Could not read image: {image_path}")

        # Resize to optimal width (1200px for speed vs accuracy balance)
        height, width = img.shape[:2]
        if width != 1200:
            scale = 1200 / width
            img = cv2.resize(img, (1200, int(height * scale)), interpolation=cv2.INTER_LINEAR)

        # Quick denoise
        denoised = cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 15)
        img_rgb = cv2.cvtColor(denoised, cv2.COLOR_BGR2RGB)

        return img_rgb

    def extract_text_fast(self, image_path: str) -> str:
        """Fast text extraction using pytesseract"""
        img_rgb = self.preprocess_fast(image_path)
        pil_image = Image.fromarray(img_rgb)

        # Optimized tesseract config for speed
        custom_config = r'--oem 3 --psm 6'
        text = pytesseract.image_to_string(pil_image, config=custom_config)
        return text.strip()

    def identify_document_type(self, text: str) -> str:
        """Quickly identify if document is Aadhaar or PAN"""
        text_upper = text.upper()
        aadhaar_score = sum(1 for keyword in self.aadhaar_keywords if keyword in text_upper)
        pan_score = sum(1 for keyword in self.pan_keywords if keyword in text_upper)

        if re.search(r'\b\d{4}\s?\d{4}\s?\d{4}\b', text):
            aadhaar_score += 2

        if re.search(r'\b[A-Z]{5}\d{4}[A-Z]\b', text_upper):
            pan_score += 2

        if aadhaar_score > pan_score:
            return "AADHAAR"
        elif pan_score > aadhaar_score:
            return "PAN"
        else:
            return "UNKNOWN"

    def extract_aadhaar_number(self, text: str) -> Optional[str]:
        pattern = r'\b(\d{4})\s?(\d{4})\s?(\d{4})\b'
        match = re.search(pattern, text)
        if match:
            return f"{match.group(1)} {match.group(2)} {match.group(3)}"
        return None

    def extract_pan_number(self, text: str) -> Optional[str]:
        text_upper = text.upper()
        pattern = r'\b[A-Z]{5}\d{4}[A-Z]\b'
        match = re.search(pattern, text_upper)
        return match.group(0) if match else None

    def extract_name(self, text: str, doc_type: str) -> Optional[str]:
        lines = text.split('\n')
        exclude_words = {
            'GOVERNMENT', 'INDIA', 'AADHAAR', 'DOB', 'MALE', 'FEMALE',
            'INCOME', 'TAX', 'DEPARTMENT', 'PAN', 'PERMANENT', 'ACCOUNT',
            'NUMBER', 'CARD', 'UNIQUE', 'IDENTIFICATION', 'AUTHORITY',
            'OF', 'THE', 'TO', 'ADDRESS', 'FATHER', 'NAME', 'SIGNATURE'
        }

        candidates = []
        for line in lines:
            words = line.strip().split()
            name_parts = []
            for word in words:
                word_clean = re.sub(r'[^A-Za-z]', '', word)
                if (len(word_clean) > 2 and word_clean.upper() not in exclude_words and word_clean.isalpha()):
                    name_parts.append(word_clean.upper())
                else:
                    if len(name_parts) >= 2:
                        candidates.append(' '.join(name_parts))
                    name_parts = []
            if len(name_parts) >= 2:
                candidates.append(' '.join(name_parts))

        return max(candidates, key=len) if candidates else None

    def extract_dob(self, text: str) -> Optional[str]:
        patterns = [
            r'\b(\d{2})[/-](\d{2})[/-](\d{4})\b',
            r'\b(\d{2})\s+(\d{2})\s+(\d{4})\b',
            r'DOB[:\s]+(\d{2})[/-](\d{2})[/-](\d{4})',
            r'YOB[:\s]+(\d{4})'
        ]
        for pattern in patterns:
            match = re.search(pattern, text)
            if match:
                if len(match.groups()) == 3:
                    return f"{match.group(1)}/{match.group(2)}/{match.group(3)}"
                elif len(match.groups()) == 1:
                    return f"Year: {match.group(1)}"
        return None

    def extract_gender(self, text: str) -> Optional[str]:
        text_upper = text.upper()
        if 'FEMALE' in text_upper or 'महिला' in text or 'स्त्री' in text:
            return 'FEMALE'
        elif 'MALE' in text_upper or 'पुरुष' in text:
            return 'MALE'
        return None

    def extract_address(self, text: str) -> Optional[str]:
        lines = text.split('\n')
        address_start_keywords = ['ADDRESS', 'S/O', 'D/O', 'C/O', 'पता']
        address_lines = []
        capture = False

        for i, line in enumerate(lines):
            line_upper = line.upper()
            if any(keyword in line_upper for keyword in address_start_keywords):
                capture = True
                if len(line.strip()) > 10:
                    address_lines.append(line.strip())
                continue

            if capture and line.strip():
                if re.search(r'\b\d{4}\s?\d{4}\s?\d{4}\b', line) or re.search(r'\b[A-Z]{5}\d{4}[A-Z]\b', line):
                    break
                address_lines.append(line.strip())
                if len(address_lines) > 5: break

        if address_lines:
            return re.sub(r'\s+', ' ', ' '.join(address_lines))
        return None

    def extract_mobile(self, text: str) -> Optional[str]:
        patterns = [r'\b([6-9]\d{9})\b', r'\+91[\s-]?([6-9]\d{9})\b', r'MOBILE[:\s]+([6-9]\d{9})']
        for pattern in patterns:
            match = re.search(pattern, text)
            if match: return match.group(1) if len(match.groups()) > 0 else match.group(0)
        return None

    def calculate_age(self, dob: str) -> Optional[int]:
        if not dob: return None
        try:
            year_match = re.search(r'(\d{4})', dob)
            if year_match:
                age = 2025 - int(year_match.group(1))
                return age if 0 < age < 120 else None
        except: pass
        return None

    def extract_info(self, image_path: str) -> Dict:
        text = self.extract_text_fast(image_path)
        if not text or len(text) < 50:
            return {"status": "ERROR", "message": "Insufficient text extracted"}

        doc_type = self.identify_document_type(text)
        if doc_type == "UNKNOWN":
            return {"status": "ERROR", "message": "Document not recognized", "text": text[:100]}

        result = {
            "status": "SUCCESS",
            "document_type": doc_type,
            "name": self.extract_name(text, doc_type),
            "dob": self.extract_dob(text),
            "gender": self.extract_gender(text),
            "address": self.extract_address(text),
            "mobile": self.extract_mobile(text),
        }

        if result["dob"]: result["age"] = self.calculate_age(result["dob"])
        if doc_type == "AADHAAR": result["aadhaar_number"] = self.extract_aadhaar_number(text)
        elif doc_type == "PAN": result["pan_number"] = self.extract_pan_number(text)

        return result

In [20]:
from google.colab import files

def main():
    extractor = EKYCExtractor()
    print("Please upload your Aadhaar or PAN card image (JPG, PNG):")
    uploaded = files.upload()

    for filename in uploaded.keys():
        print(f"\n{'='*40}")
        print(f"Processing: {filename}")
        print(f"{'='*40}")

        result = extractor.extract_info(filename)

        if result.get('status') == 'SUCCESS':
            print(f"📄 DOCUMENT TYPE: {result.get('document_type')}\n")

            fields = [
                ('Name', 'name'), ('Date of Birth', 'dob'), ('Age', 'age'),
                ('Gender', 'gender'), ('Address', 'address'), ('Mobile', 'mobile'),
                ('Aadhaar Number', 'aadhaar_number'), ('PAN Number', 'pan_number')
            ]

            for label, key in fields:
                value = result.get(key)
                if value:
                    print(f"{label:20s}: {value}")
        else:
            print(f"❌ Error: {result.get('message')}")

if __name__ == "__main__":
    main()

Please upload your Aadhaar or PAN card image (JPG, PNG):


Saving adhar.jfif to adhar (2).jfif

Processing: adhar (2).jfif
📄 DOCUMENT TYPE: AADHAAR

Name                : SOLAPUR CAY SOLAPUR NORTH SOLAPUR
Date of Birth       : 21/10/2906
Gender              : MALE
Aadhaar Number      : 5296 9089 2168
